In [1]:
import os
import csv
import re

In [6]:
def extract_mean_std_from_qdp(qdp_path):
    """
    Extract mean and standard deviation from a .qdp file with known format.
    """
    mean_period = None
    std_period = None
    
    with open(qdp_path, 'r') as file:
        for line in file:
            if "Mean period:" in line:
                match = re.search(r"Mean period:\s*([\d\.Ee+-]+)", line)
                if match:
                    mean_period = float(match.group(1))
            elif "Standard deviation" in line:
                match = re.search(r"Standard deviation.*?:\s*([\d\.Ee+-]+)", line)
                if match:
                    std_period = float(match.group(1))
    
    return mean_period, std_period

def traverse_and_collect(root_dir, output_csv):
    rows = []

    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.qdp'):
                obs_id = os.path.basename(root)
                qdp_path = os.path.join(root, file)
                mean, std = extract_mean_std_from_qdp(qdp_path)
                if mean is not None and std is not None:
                    rows.append((obs_id, mean, std))
    
    rows.sort(key=lambda x: int(x[0]))
    
    # Save to CSV
    with open(output_csv, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['ObsID', 'Period', 'Period_Error'])
        writer.writerows(rows)


In [8]:

# === USAGE ===
root_folder = "./bootstrap_error_uncor_lc"  # Change this to your actual directory
output_csv_path = "bootstrap_error_uncor_lc/error_estimate_taken.csv"

traverse_and_collect(root_folder, output_csv_path)
print(f"Saved summary to: {output_csv_path}")

Saved summary to: bootstrap_error_uncor_lc/error_estimate_taken.csv
